<a href="https://colab.research.google.com/github/Yazmaz98/data-stream/blob/master/Batch_Time_Series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import numpy as np


# create dataset
def create_dataset(X : np.array, n_samples : int, mode : str='direct'):
    """
    Function takes time-series X as an input and provides time-series Xo and 
    the test data yo for output.
    ----------------------------------
    n_samples : int, number of samples
    mode : string,
    - 'direct': X0 are the generated batches, yo is x[t+1]
    - 'log': X0 - outputs the log-returns of the data, yo is the next log-return.
    ----------------------------------
    X0: np.array, shape: (len(X) - n_samples - 1, n_samples)
    y0: np.array, shape: len(X) - n_samples - 1
    """
    if mode == 'direct':
        X0 = np.zeros((len(X) - n_samples - 1, n_samples))
        y0 = np.zeros(len(X) - n_samples - 1)
        for i in range(len(X) - n_samples - 1):
            X0[i] = X[i : i + n_samples]
            y0[i] = X[i + n_samples + 1]
    elif mode == 'log':
        X0 = np.zeros((len(X) - n_samples - 2, n_samples))
        y0 = np.zeros(len(X) - n_samples - 2)
        X_return = np.log(X[1:] / X[:-1])
        for i in range(len(X) - n_samples - 2):
            X0[i] = X_return[i:i + n_samples]
            y0[i] = X_return[i + n_samples + 1]
    else:
        raise(NotImplementedError("Unknown method"))

    return X0, y0

In [4]:
# generate test data
n = 100
X = 1 + np.random.rand(n)
X_return = np.log(X[1:] / X[:-1])
X_return[0: 50]

# Xo, yo = create_dataset(X, 50, mode='lq')
# for x, y in zip(xo, yo):
#   print(x, y)
#   break

array([ 0.47387691, -0.09230929, -0.44070218, -0.02194789,  0.57264823,
       -0.41045353,  0.24620112, -0.09651745,  0.26585863, -0.03730824,
       -0.44810634,  0.07284804,  0.11949079, -0.36532343,  0.03112855,
        0.12612678,  0.18319591,  0.04235503,  0.20271476,  0.02156988,
       -0.44519878,  0.24399109, -0.14448438, -0.16750146,  0.33285557,
       -0.28919009, -0.14295848,  0.60004176, -0.16652167, -0.42873055,
        0.17043171,  0.11485436,  0.08963303, -0.10327665,  0.07900806,
       -0.30575239,  0.53473429,  0.00085213, -0.30897029,  0.26870502,
       -0.52300607,  0.49961655, -0.28715534, -0.06679381,  0.40241681,
       -0.06734385, -0.02895494, -0.0595015 ,  0.23239301, -0.08279692])

In [10]:
!pip install pmdarima

     |████████████████████████████████| 1.4 MB 4.2 MB/s 
     |████████████████████████████████| 9.8 MB 50.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [9]:
import numpy as np

from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm

class ARIMA_model():
    def __init__(self, p=None, q=None, r=None):
        """
        p, q, r: params of ARIMA model,
        see https://www.statsmodels.org/devel/generated/statsmodels.tsa.arima.model.ARIMA.html
        """
        self.p = p
        self.q = q
        self.r = r

    def get_parameters(X):
        ## Adf Test
        ndiffs(X, test='adf')  # 2

        # KPSS test
        ndiffs(X, test='kpss')  # 0

        # PP test:
        ndiffs(X, test='pp')  # 2

        return p, q, r
    
    def fit_predict(self, X, y, get_params=True):
        if get_params:
          self.p, self.q, self.r = get_parameters(X)
          
        y_pred = np.zeros_like(y)
        for i in tqdm(range(len(X))):
            ts = X[i]
            model = ARIMA(ts, order=(self.p, self.q, self.r))
            model_fit = model.fit()
            y_pred[i] = model_fit.forecast()[0][0]
        return y_pred

from sklearn.metrics import mean_squared_error as MSE

n_samples = 70
p, q, r = 5, 0, 1

model = ARIMA_model(p, q, r)
X_train, y_train = create_dataset(X, n_samples)
y_pred = model.fit_predict(X_train, y_train)
print()
print("MSE: %.5f" %MSE(y_train, y_pred))

ModuleNotFoundError: ignored

In [ ]:
4, 0, 1: 0.12670
5, 0, 1: 0.08793

In [7]:
## Adf Test
ndiffs(X, test='adf')  # 2

# KPSS test
ndiffs(X, test='kpss')  # 0

# PP test:
ndiffs(X, test='pp')  # 2

NameError: ignored

In [112]:
y_pred

array([1.46108826, 1.29886672, 1.54681982, 1.43419192, 1.54998292,
       1.39449436, 1.61371162, 1.30226069, 1.65769368, 1.46848136,
       1.50378828, 1.55831189, 1.50489814, 1.42443501, 1.43845222,
       1.61059161, 1.50409444, 1.34187683, 1.45380301, 1.35737642,
       1.59438204, 1.39764124, 1.56674718, 1.360295  , 1.40259733,
       1.43905342, 1.45829277, 1.31574274, 1.34695011])

In [113]:
y_train

array([1.1809162 , 1.6521392 , 1.14848864, 1.68111783, 1.00594754,
       1.99967502, 1.7134624 , 1.05229522, 1.20694885, 1.25287978,
       1.05938383, 1.0039687 , 1.34377103, 1.87261427, 1.58202245,
       1.63972114, 1.24581597, 1.98284274, 1.15003742, 1.25611585,
       1.8530457 , 1.63688331, 1.99829534, 1.91195651, 1.33632171,
       1.01809234, 1.82768771, 1.17213212, 1.42742301])